In [403]:
import pandas as pd
import matplotlib.pyplot as py
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from nltk.util import ngrams
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chethana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/chethana/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [421]:
df=pd.read_csv("output3_northwestern.csv")
# df_kellog_2 = pd.read_csv("faculty_data.csv")
# df_kellog_1= pd.read_csv("KELLOGfaculty1.csv")

In [374]:
print(df.shape)
df

(635, 5)


In [375]:
df.isnull().sum()

Name                       0
Research Interest          6
Selected Publications      6
Significant Recognition    0
University                 0
dtype: int64

In [376]:
df=df.dropna(subset=['Research Interest'])
mask = df['Research Interest'].str.contains('not found', case=False, na=False)
df=df[~mask]

print(df.shape)
df.head()

(280, 5)


,Name,Research Interest,Selected Publications,Significant Recognition,University
0,Daniel Abrams,Nonlinear dynamics; physics of social systems;...,"1) J.D. Johnson, A.M. Redlich, and D.M. Ab...","James S. McDonnell Foundation Scholar, 2010-20...",Northwestern University
1,Hayim Abreveya,Hayim Abrevayais the Senior Manager for Explor...,"Hayim Abrevaya, Parag Jain, Monomolecular Crac...",Significant recognition not found on the webpage.,Northwestern University
3,Ankit Agrawal,Prof. Ankit Agrawal specializes in interdiscip...,"McCormick School of Engineering, Northwestern ...",Significant recognition not found on the webpage.,Northwestern University
6,Ange-Therese Akono,Dr. Akono’s laboratory investigates fracture a...,"McCormick School of Engineering, Northwestern ...",NIH Diversity Grant (2020) NIH Building Up Sch...,Northwestern University
9,Emma Alexander,"I am interested in low-level, bio-inspired, ph...","McCormick School of Engineering, Northwestern ...",Significant recognition not found on the webpage.,Northwestern University


In [377]:
df.isnull().sum()

Name                       0
Research Interest          0
Selected Publications      6
Significant Recognition    0
University                 0
dtype: int64

In [404]:
import spacy
from spacy import displacy

def replace_named_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    
    # Replace named entities with their labels
    replaced_tokens = [ent.ent_type_ if ent.ent_type_ else ent.text for ent in doc]
    
    return ' '.join(replaced_tokens)

nltk.download('wordnet')
from spellchecker import SpellChecker
nlp = spacy.load("en_core_web_sm")
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    text = re.sub(r'\b\d+\b', '', text)  # Remove numerical values
    
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    bigrams = list(ngrams(tokens, 2))
    trigrams = list(ngrams(tokens, 3))
    
    total_tokens = tokens + [' '.join(bigram) for bigram in bigrams] + [' '.join(trigram) for trigram in trigrams]
    
    # spell = SpellChecker()
    # corrected_tokens = [spell.correction(word) if word is not None else None for word in total_tokens]  # Replace None values
    corrected_tokens = [word for word in total_tokens if word is not None]  # Exclude None values
    corrected_text = ' '.join(corrected_tokens)
    
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in word_tokenize(corrected_text)]
    
    # Replace named entities using the replace_named_entities function
    replaced_entities_text = replace_named_entities(' '.join(lemmatized_tokens))
    
    doc = nlp(replaced_entities_text)
    
    # Remove tokens identified as stopwords, adverbs, and proper nouns (names)
    replaced_entities_tokens = [token.text if token.pos_ not in ['VERB', 'ADV', 'PROPN', 'ADJ'] and token.text.lower() not in stop_words else '' for token in doc]
    replaced_entities_text = ' '.join(replaced_entities_tokens)
    
    keywords = [word for word in word_tokenize(replaced_entities_text) if len(word) > 2]
    preprocessed_text = ' '.join(keywords)
    
    return preprocessed_text

# Example usage:
df['Preprocessed Research Interest'] = df['Research Interest'].fillna('').apply(preprocess_text)
#df.head()


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/chethana/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [384]:
import spacy
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from nltk import ngrams
import pandas as pd
from tqdm import tqdm


# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Function for tokenization using SpaCy
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc if not token.is_stop and token.is_alpha]

# Function for document embedding using Word2Vec
def document_embedding(tokens, model):
    valid_tokens = [token for token in tokens if token in model.wv]
    
    if not valid_tokens:
        return None
    
    # Use the mean of word embeddings with attention weights
    attention_weights = [1.0] * len(valid_tokens)
    doc_embedding = sum(model.wv[token] * weight for token, weight in zip(valid_tokens, attention_weights)) / sum(attention_weights)
    
    return doc_embedding

# Assuming df is your DataFrame with a 'Preprocessed Research Interest' column
#corpus = df['Preprocessed Research Interest'].apply(tokenize).tolist()
#corpus = df['Preprocessed Research Interest'].apply(preprocess_text).apply(tokenize).tolist()
print(df.columns)

tqdm.pandas(desc="Preprocessing and Tokenizing")
corpus = df['Preprocessed Research Interest'].progress_apply(preprocess_text).progress_apply(tokenize).tolist()


# Train Word2Vec model
model = Word2Vec(sentences=corpus, vector_size=100, window=4, min_count=1, workers=5)
tqdm.pandas(desc="Embedding Documents", total=len(df))

# Apply document embedding to each document in the DataFrame
df['Doc_Embedding'] = df['Preprocessed Research Interest'].progress_apply(lambda x: document_embedding(tokenize(x), model))

# Drop rows where document embedding couldn't be computed
df = df.dropna(subset=['Doc_Embedding'])

# Convert document embeddings to a matrix for clustering
X = pd.DataFrame(df['Doc_Embedding'].tolist())

# Apply KMeans clustering
kmeans = KMeans(n_clusters=40, random_state=42)
df['Cluster_Word2Vec'] = kmeans.fit_predict(X)

# Analyze the resulting clusters
clustered_data = df.groupby('Cluster_Word2Vec')['Preprocessed Research Interest'].apply(list)
for cluster_label, research_interests in tqdm(clustered_data.items(), desc="Analyzing Clusters", total=len(clustered_data)):
    print(f"Cluster {cluster_label}: {', '.join(research_interests)}")

# Assuming you have a trained vectorizer
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(df['Preprocessed Research Interest'])

# Example usage of the new TF-IDF vectorizer
new_text = "I am interested in working on swarm robots. I find it cool."
preprocessed_new_text = preprocess_text(new_text)
X_new_tfidf = vectorizer.transform([preprocessed_new_text])

# Print corresponding words and TF-IDF scores
feature_names = vectorizer.get_feature_names_out()
print("Corresponding words and TF-IDF scores:")
for word_index, tfidf_score in zip(X_new_tfidf.indices, X_new_tfidf.data):
    word = feature_names[word_index]
    print(f"Word: {word}, TF-IDF Score: {tfidf_score}")


# for cluster_label, research_interests in clustered_data.items():
#     print(f"Cluster {cluster_label}: {', '.join(research_interests)}")


Index(['Name', 'Research Interest', 'Selected Publications',
       'Significant Recognition', 'University',
       'Preprocessed Research Interest'],
      dtype='object')


Embedding Documents: 100%|██████████| 280/280 [00:07<00:00, 38.89it/s]
/Users/chethana/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
Analyzing Clusters: 100%|██████████| 40/40 [00:00<00:00, 97541.95it/s]


Cluster 0: PERSON PERSON research envisions material thing approach lifecycle batteryfree system operation endofthelifecycle disposalreuse intersection device fabrication lowpower system design work application domain like home health accessibility biodiversity infrastructure monitoring aroras research research envisions envisions material material thing thing approach approach lifecycle lifecycle materialsmanufacturing batteryfree batteryfree system system operation operation endofthelifecycle disposalreuse work intersection intersection device device fabrication fabrication lowpower lowpower system system design design work application application domain domain like like home health accessibility accessibility biodiversity biodiversity infrastructure infrastructure monitoring prof research research envisions research envisions envisions material material material thing thing thing sustainabilityfirst approach approach approach lifecycle lifecycle batteryfree materialsmanufacturing ba

In [411]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

# Assuming df is your DataFrame with a 'Preprocessed Research Interest' column
corpus = df['Preprocessed Research Interest'].apply(word_tokenize).tolist()

# Train Word2Vec model
model = Word2Vec(sentences=corpus, vector_size=100, window=4, min_count=1, workers=5)
word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}

# Function to get document embedding using Word2Vec
def document_embedding(document, embeddings):
    words = document.split()
    valid_words = [word for word in words if word in embeddings]
    
    if not valid_words:
        return None
    
    # Use the mean of word embeddings as the document embedding
    doc_embedding = sum(embeddings[word] for word in valid_words) / len(valid_words)
    return doc_embedding

In [415]:

# new_text = "I am interested in working on swarm robots. I find it cool."
# preprocessed_new_text = preprocess_text(new_text)
# new_text_embedding = document_embedding(tokenize(preprocessed_new_text), model)

# X_new_embedding = new_text_embedding.reshape(1, -1)
# predicted_cluster = kmeans.predict(X_new_embedding)[0]
# print(f"The new text belongs to Cluster {predicted_cluster}")
def document_embedding(tokens, model):
    valid_tokens = [token for token in tokens if token in model.wv]

    if not valid_tokens:
        return None

    # Use the mean of word embeddings with attention weights
    attention_weights = [1.0] * len(valid_tokens)
    doc_embedding = sum(model.wv[token] * weight for token, weight in zip(valid_tokens, attention_weights)) / sum(attention_weights)

    return doc_embedding

# Example usage with new text
new_text = "I am interested in working on swarm robots. I find it cool."
preprocessed_new_text = preprocess_text(new_text)
new_text_tokens = tokenize(preprocessed_new_text)
new_text_embedding = document_embedding(new_text_tokens, model)

if new_text_embedding is not None:
    X_new_embedding = np.array(new_text_embedding).reshape(1, -1)
    predicted_cluster = kmeans.predict(X_new_embedding)[0]
    print(f"The new text belongs to Cluster {predicted_cluster}")
else:
    print("No valid tokens found for the new text.")


The new text belongs to Cluster 5


In [398]:
# # kmeans = KMeans(n_clusters=10, random_state=42)
# # df['Cluster_TFIDF'] = kmeans.fit_predict(X_tfidf)

# # Apply document embedding to each document in the DataFrame
# df['Doc_Embedding'] = df['Preprocessed Research Interest'].apply(lambda x: document_embedding(x, word_embeddings))

# # Drop rows where document embedding couldn't be computed
# df = df.dropna(subset=['Doc_Embedding'])

# # Convert document embeddings to a matrix for clustering
# X = pd.DataFrame(df['Doc_Embedding'].tolist())

# # Apply KMeans clustering
# kmeans = KMeans(n_clusters=65, random_state=42)
# df['Cluster_Word2Vec'] = kmeans.fit_predict(X)

# # Analyze the resulting clusters
# clustered_data = df.groupby('Cluster_Word2Vec')['Preprocessed Research Interest'].apply(list)

# for cluster_label, research_interests in clustered_data.items():
#     print(f"Cluster {cluster_label}: {', '.join(research_interests)}")

In [386]:
df.head()

,Name,Research Interest,Selected Publications,Significant Recognition,University,Preprocessed Research Interest,Doc_Embedding,Cluster_Word2Vec
0,Daniel Abrams,Nonlinear dynamics; physics of social systems;...,"1) J.D. Johnson, A.M. Redlich, and D.M. Ab...","James S. McDonnell Foundation Scholar, 2010-20...",Northwestern University,physic system geoscience pattern formation osc...,"[-0.967583, 0.23978923, -0.2583162, 0.06396913...",4
1,Hayim Abreveya,Hayim Abrevayais the Senior Manager for Explor...,"Hayim Abrevaya, Parag Jain, Monomolecular Crac...",Significant recognition not found on the webpage.,Northwestern University,manager research uop supplier catalyst process...,"[-0.3425098, 0.015802791, -0.18179423, 0.15367...",37
3,Ankit Agrawal,Prof. Ankit Agrawal specializes in interdiscip...,"McCormick School of Engineering, Northwestern ...",Significant recognition not found on the webpage.,Northwestern University,intelligence data analytics via performance da...,"[-0.292008, 0.47452474, -0.39404055, 0.0916809...",38
6,Ange-Therese Akono,Dr. Akono’s laboratory investigates fracture a...,"McCormick School of Engineering, Northwestern ...",NIH Diversity Grant (2020) NIH Building Up Sch...,Northwestern University,PERSON PERSON PERSON investigates failure mech...,"[-0.5196798, 0.44082066, -0.15215804, 0.091786...",35
9,Emma Alexander,"I am interested in low-level, bio-inspired, ph...","McCormick School of Engineering, Northwestern ...",Significant recognition not found on the webpage.,Northwestern University,lowlevel computer vision understanding animal ...,"[-0.31366366, 0.20658588, -0.853089, 0.244482,...",20


In [387]:
for cluster_id in range(50):
    cluster_df = df[df['Cluster_Word2Vec'] == cluster_id]
    print(f"\nCluster {cluster_id}:")
    print(cluster_df['Preprocessed Research Interest'].head())


Cluster 0:
22    PERSON PERSON research envisions material thin...
26    research program metamaterials device lab area...
32    research wave biomechancs structure research g...
35    energyefficient computing communication techno...
85    research interest manufacturing process system...
Name: Preprocessed Research Interest, dtype: object

Cluster 1:
46     school university
208    school university
241    school university
259    school university
308    school university
Name: Preprocessed Research Interest, dtype: object

Cluster 2:
99     design uncertainty machine science engineering...
426    area research optimization application machine...
584    optimization source software implementation ap...
Name: Preprocessed Research Interest, dtype: object

Cluster 3:
103    research interest focus intelligence data scie...
141    PERSON PERSON professor department computer sc...
159    research interest swarm robotics robot manipul...
177    PERSON forbus walter professor computer sc

In [390]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering

model = SentenceTransformer('bert-base-nli-mean-tokens')

document_embeddings = model.encode(df['Preprocessed Research Interest'])


hierarchical_clustering = AgglomerativeClustering(n_clusters=10)  
df['Cluster_BERT'] = hierarchical_clustering.fit_predict(document_embeddings)


In [389]:
if 210 in df.index:
    print("Index 156 exists.")
else:
    print("Index 156 does not exist.")
df = df.reset_index(drop=True)
if 210 in df.index:
    print("Index 156 exists.")
else:
    print("Index 156 does not exist.")


Index 156 does not exist.
Index 156 exists.


In [325]:
# for cluster_id in range(50):
#     cluster_df = df[df['Cluster_BERT'] == cluster_id]
#     print(f"\nCluster {cluster_id}:")
#     print(cluster_df['Preprocessed Research Interest'].head())

In [391]:
df.head()

,Name,Research Interest,Selected Publications,Significant Recognition,University,Preprocessed Research Interest,Doc_Embedding,Cluster_Word2Vec,Cluster_BERT
0,Daniel Abrams,Nonlinear dynamics; physics of social systems;...,"1) J.D. Johnson, A.M. Redlich, and D.M. Ab...","James S. McDonnell Foundation Scholar, 2010-20...",Northwestern University,physic system geoscience pattern formation osc...,"[-0.967583, 0.23978923, -0.2583162, 0.06396913...",4,1
1,Hayim Abreveya,Hayim Abrevayais the Senior Manager for Explor...,"Hayim Abrevaya, Parag Jain, Monomolecular Crac...",Significant recognition not found on the webpage.,Northwestern University,manager research uop supplier catalyst process...,"[-0.3425098, 0.015802791, -0.18179423, 0.15367...",37,6
2,Ankit Agrawal,Prof. Ankit Agrawal specializes in interdiscip...,"McCormick School of Engineering, Northwestern ...",Significant recognition not found on the webpage.,Northwestern University,intelligence data analytics via performance da...,"[-0.292008, 0.47452474, -0.39404055, 0.0916809...",38,2
3,Ange-Therese Akono,Dr. Akono’s laboratory investigates fracture a...,"McCormick School of Engineering, Northwestern ...",NIH Diversity Grant (2020) NIH Building Up Sch...,Northwestern University,PERSON PERSON PERSON investigates failure mech...,"[-0.5196798, 0.44082066, -0.15215804, 0.091786...",35,1
4,Emma Alexander,"I am interested in low-level, bio-inspired, ph...","McCormick School of Engineering, Northwestern ...",Significant recognition not found on the webpage.,Northwestern University,lowlevel computer vision understanding animal ...,"[-0.31366366, 0.20658588, -0.853089, 0.244482,...",20,0


In [420]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from nltk import word_tokenize, pos_tag
X = np.stack(df['Doc_Embedding'].to_numpy())
y = df['Cluster_Word2Vec']
X_train, X_test, y_train, y_test = train_test_split(X, df['Cluster_Word2Vec'], test_size=0.2, random_state=42)

classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# def document_embedding(text, model):
#     return model.encode([text])[0]
new_text = "I am interested in working on swarm robots. I find it cool."
preprocessed_new_text = preprocess_text(new_text)
new_text_embedding = document_embedding(tokenize(preprocessed_new_text), model)
X_new_embedding = new_text_embedding.reshape(1, -1)
predicted_cluster = kmeans.predict(X_new_embedding)[0]
df['Cluster_Word2Vec'] = kmeans.fit_predict(X)
y = df['Cluster_Word2Vec']
#classifier.fit(X, y)
print("kmeans prediction cluster is", predicted_cluster)

classifier.fit(X_train, y_train)
predicted_cluster = classifier.predict(X_new_embedding)[0]
print("RandomForestClassifier prediction cluster is", predicted_cluster)


print("Corresponding words and TF-IDF scores:")
for word_index, tfidf_score in zip(X_new_tfidf.indices, X_new_tfidf.data):
    word = feature_names[word_index]
    print(f"Word: {word}, TF-IDF Score: {tfidf_score}")
cluster_df = df[df['Cluster_Word2Vec'] == predicted_cluster] 
cluster_df['Awards'] = cluster_df['Significant Recognition'].apply(lambda x: 0 if 'not found' in x else int(x.split()[0]) if x.split()[0].isdigit() else 0)
cluster_df['Publications'] = cluster_df['Selected Publications'].str.count(',') + 1

sorted_professors = cluster_df.sort_values(by=['Awards', 'Publications'], ascending=False).head(5)

print(f"Cluster Name: {predicted_cluster}")
print("Top Professors:")
for index, row in sorted_professors.iterrows():
    print(f"Professor: {row['Name']}, University: {row['University']}, Awards: {row['Awards']}, Publications: {row['Publications']}")


/Users/chethana/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


kmeans prediction cluster is 5
RandomForestClassifier prediction cluster is 35
Corresponding words and TF-IDF scores:
Word: swarm, TF-IDF Score: 0.8417557395784211
Word: robot, TF-IDF Score: 0.5398585693371788
Cluster Name: 35
Top Professors:
Professor: Kuniaki Dohda, University: Northwestern University, Awards: 0, Publications: 35.0
Professor: Seth Lichter, University: Northwestern University, Awards: 0, Publications: 12.0
Professor: Zhe Ji, University: Northwestern University, Awards: 0, Publications: 11.0
Professor: Michael Bedzyk, University: Northwestern University, Awards: 0, Publications: 10.0
Professor: Ludmilla Aristilde, University: Northwestern University, Awards: 0, Publications: 9.0


/var/folders/_6/p3xzjb897kg8d2f55c7664xc0000gn/T/ipykernel_54829/3755470505.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_df['Awards'] = cluster_df['Significant Recognition'].apply(lambda x: 0 if 'not found' in x else int(x.split()[0]) if x.split()[0].isdigit() else 0)
/var/folders/_6/p3xzjb897kg8d2f55c7664xc0000gn/T/ipykernel_54829/3755470505.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cluster_df['Publications'] = cluster_df['Selected Publications'].str.count(',') + 1


In [419]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from nltk import word_tokenize, pos_tag
import numpy as np

# Assuming you have a DataFrame df with 'Doc_Embedding' and 'Cluster_Word2Vec' columns
X = np.stack(df['Doc_Embedding'].to_numpy())
y = df['Cluster_Word2Vec']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the RandomForestClassifier on the training set
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy}")
# def document_embedding(text, model):
#     return model.encode([text])[0]
# Now, you can use the trained classifier for predictions on new data
new_text = "swarm robots"
preprocessed_new_text = preprocess_text(new_text)
new_text_embedding = document_embedding(tokenize(preprocessed_new_text), model)
X_new_embedding = new_text_embedding.reshape(1, -1)
predicted_cluster = classifier.predict(X_new_embedding)[0]
print("RandomForestClassifier prediction is, cluster no ", predicted_cluster)



Accuracy on the test set: 0.625
RandomForestClassifier prediction is, cluster no  37


In [417]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score

# Assuming you have a DataFrame df with 'Doc_Embedding' and 'Cluster_Word2Vec' columns
X = np.stack(df['Doc_Embedding'].to_numpy())
y = df['Cluster_Word2Vec']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the RandomForestClassifier on the training set
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy}")

# Now, you can use the trained classifier for predictions on new data
new_text = "swarm robots"
preprocessed_new_text = preprocess_text(new_text)
new_text_embedding = document_embedding(tokenize(preprocessed_new_text), model)
X_new_embedding = new_text_embedding.reshape(1, -1)
predicted_cluster = classifier.predict(X_new_embedding)[0]
print("RandomForestClassifier prediction is, cluster no ", predicted_cluster)


Accuracy on the test set: 0.625
RandomForestClassifier prediction is, cluster no  37


In [343]:
# Get accuracy on the training set
train_accuracy = classifier.score(X_train, y_train)
print(f"Training Accuracy: {train_accuracy}")

# Get accuracy on the test set
test_accuracy = classifier.score(X_test, y_test)
test_accuracy

Training Accuracy: 1.0


0.5714285714285714

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from nltk import word_tokenize, pos_tag
import numpy as np
from sklearn.metrics import accuracy_score


# Assuming you have a DataFrame df with 'Doc_Embedding' and 'Cluster_Word2Vec' columns
X = np.stack(df['Doc_Embedding'].to_numpy())
y = df['Cluster_Word2Vec']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the RandomForestClassifier on the training set
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy}")
# def document_embedding(text, model):
#     return model.encode([text])[0]
# Now, you can use the trained classifier for predictions on new data
new_text = "swarm robots"
preprocessed_new_text = preprocess_text(new_text)
new_text_embedding = document_embedding(tokenize(preprocessed_new_text), model)
X_new_embedding = new_text_embedding.reshape(1, -1)
predicted_cluster = classifier.predict(X_new_embedding)[0]
print("RandomForestClassifier prediction is, cluster no ", predicted_cluster)



NameError: name 'df' is not defined